In [1]:
import sqlite3
import pandas as pd
import warnings

import sys
import os

sys.path.insert(0, os.path.abspath("/home/airflow/dags"))

from lead_scoring_data_pipeline.utils import (
    build_dbs,
    load_data_into_db,
    map_city_tier,
    map_categorical_vars,
    interactions_mapping
)

from lead_scoring_data_pipeline.constants import (
    DB_FULL_PATH
)

from constants import (
    UNIT_TEST_DB_PATH,
    UNIT_TEST_DB_FILE_NAME,
    TEST_DATA_CSV_PATH
)

warnings.filterwarnings("ignore")

In [2]:
load_data_into_db(TEST_DATA_CSV_PATH)
conn = sqlite3.connect(DB_FULL_PATH)
loaded_data = pd.read_sql('SELECT * FROM loaded_data', conn)
conn_ut = sqlite3.connect(UNIT_TEST_DB_PATH + UNIT_TEST_DB_FILE_NAME)
test_case = pd.read_sql('SELECT * FROM loaded_data_test_case', conn_ut)
conn.close()
conn_ut.close()
assert test_case.equals(loaded_data), "❌ Data mismatch in load_data_into_db()"

In [3]:
map_city_tier()
conn = sqlite3.connect(DB_FULL_PATH)
output_df = pd.read_sql('SELECT * FROM city_tier_mapped', conn)
conn_ut = sqlite3.connect(UNIT_TEST_DB_PATH + UNIT_TEST_DB_FILE_NAME)
expected_df = pd.read_sql('SELECT * FROM city_tier_mapped_test_case', conn_ut)
conn.close()
conn_ut.close()
assert expected_df.equals(output_df), "❌ Data mismatch in map_city_tier()"

In [4]:
map_categorical_vars()
conn = sqlite3.connect(DB_FULL_PATH)
output_df = pd.read_sql('SELECT * FROM categorical_variables_mapped', conn)
conn_ut = sqlite3.connect(UNIT_TEST_DB_PATH + UNIT_TEST_DB_FILE_NAME)
expected_df = pd.read_sql('SELECT * FROM categorical_variables_mapped_test_case', conn_ut)
conn.close()
conn_ut.close()
assert expected_df.equals(output_df), "❌ Data mismatch in map_categorical_vars()"

In [5]:
interactions_mapping("final_output")


In [6]:
conn = sqlite3.connect(DB_FULL_PATH)


In [7]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)


In [8]:
tables

,name
0,TRAINING
1,INFERENCE
2,FEATURES
3,TARGET
4,PREDICTIONS
5,loaded_data
6,city_tier_mapped
7,categorical_variables_mapped
8,interactions_mapped
9,final_output


In [9]:
output_df = pd.read_sql('SELECT * FROM interactions_mapped', conn)


In [10]:
conn_ut = sqlite3.connect(UNIT_TEST_DB_PATH + UNIT_TEST_DB_FILE_NAME)
expected_df = pd.read_sql('SELECT * FROM interactions_mapped_test_case', conn_ut)
conn.close()
conn_ut.close()
assert expected_df.equals(output_df), "❌ Data mismatch in interactions_mapping()"
